## Evaluate Leave One Image Out analysis

In [1]:
import pathlib
import numpy as np
import pandas as pd

from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_curve, auc

In [2]:
def compute_avg_rank_and_pvalue(grouped_df):
    ranks = []
    p_values = []

    for _, group in grouped_df.groupby("Cell_UUID"):
        # Sort predicted probabilities in descending order
        sorted_probs = group.sort_values(by="Predicted_Probability", ascending=False).reset_index(drop=True)
        
        # Get the rank of the true class
        rank = sorted_probs[sorted_probs["Mitocheck_Phenotypic_Class"] == sorted_probs["Model_Phenotypic_Class"]].index[0] + 1
        
        # Get the p-value (predicted probability) of the true class
        p_value = sorted_probs.loc[rank - 1, "Predicted_Probability"]
        
        ranks.append(rank)
        p_values.append(p_value)

    # Calculate average rank and p-value for the group
    avg_rank = sum(ranks) / len(ranks)
    avg_p_value = sum(p_values) / len(p_values)
    
    # Calculate IQR and min/max within IQR for ranks
    iqr_rank = np.percentile(ranks, 75) - np.percentile(ranks, 25)
    min_iqr_rank = np.percentile(ranks, 25)
    max_iqr_rank = np.percentile(ranks, 75)
    
    # Calculate IQR and min/max within IQR for p-values
    iqr_p_value = np.percentile(p_values, 75) - np.percentile(p_values, 25)
    min_iqr_p_value = np.percentile(p_values, 25)
    max_iqr_p_value = np.percentile(p_values, 75)
    
    # Count number of comparisons
    count = len(ranks)
    
    return avg_rank, avg_p_value, min_iqr_rank, max_iqr_rank, min_iqr_p_value, max_iqr_p_value, count

In [3]:
# Set I/O
proba_dir = pathlib.Path("evaluations", "LOIO_probas")
loio_file = pathlib.Path(proba_dir, "compiled_LOIO_probabilities.tsv")

output_summary_file = pathlib.Path(proba_dir, "LOIO_summary_ranks.tsv.gz")
output_summary_phenotype_file = pathlib.Path(proba_dir, "LOIO_summary_ranks_perphenotype.tsv.gz")

In [4]:
loio_df = pd.read_csv(loio_file, sep="\t", index_col=0)

print(loio_df.shape)
loio_df.head()

(1710900, 11)


,Cell_UUID,Metadata_DNA,Mitocheck_Phenotypic_Class,Model_Feature_Type,Model_C,Model_l1_ratio,Model_type,Dataset_type,Balance_type,Model_Phenotypic_Class,Predicted_Probability
0,0003588c-7dda-470c-91da-a88479654936,LT0067_02/LT0067_02_99_77.tif,Polylobed,CP,1.0,0.5,final,no_ic,balanced,ADCCM,4.149980e-04
1,0003588c-7dda-470c-91da-a88479654936,LT0067_02/LT0067_02_99_77.tif,Polylobed,CP,0.1,0.1,final,no_ic,unbalanced,ADCCM,8.154955e-04
2,0003588c-7dda-470c-91da-a88479654936,LT0067_02/LT0067_02_99_77.tif,Polylobed,CP,0.1,0.0,shuffled_baseline,no_ic,balanced,ADCCM,4.095926e-02
3,0003588c-7dda-470c-91da-a88479654936,LT0067_02/LT0067_02_99_77.tif,Polylobed,CP,0.1,1.0,shuffled_baseline,no_ic,unbalanced,ADCCM,2.673641e-02
4,0003588c-7dda-470c-91da-a88479654936,LT0067_02/LT0067_02_99_77.tif,Polylobed,CP,1.0,0.5,final,no_ic,balanced,Anaphase,5.038893e-07


In [5]:
phenotype_classes = loio_df.Mitocheck_Phenotypic_Class.unique().tolist()
phenotype_classes

['Polylobed',
 'Interphase',
 'MetaphaseAlignment',
 'Prometaphase',
 'OutOfFocus',
 'Large',
 'ADCCM',
 'Elongated',
 'Apoptosis',
 'Binuclear',
 'SmallIrregular',
 'Hole',
 'Grape',
 'Metaphase',
 'Anaphase']

## Get average ranks and p value of correct prediction

- Per Image
- Per model type (final vs. shuffled)
- Per illumination correction function (IC vs. No-IC)
- Phenotype
- Feature Space

In [6]:
rank_groups = [
    "Metadata_DNA",
    "Model_type",
    "Dataset_type",
    "Mitocheck_Phenotypic_Class",
    "Model_Feature_Type",
    "Balance_type"
]

# Output data columns
output_data_columns = [
    "Average_Rank",
    "Average_P_Value",
    "Min_IQR_Rank",
    "Max_IQR_Rank", 
    "Min_IQR_P_Value",
    "Max_IQR_P_Value", 
    "Count"
]

avg_ranks = (
    loio_df.groupby(rank_groups)
    .apply(compute_avg_rank_and_pvalue)
    .reset_index()
)

avg_ranks.columns = rank_groups + ["Average_Scores"]

loio_scores_df = (
    pd.concat([
        avg_ranks.drop(columns="Average_Scores"),
        pd.DataFrame(
            avg_ranks.Average_Scores.tolist(),
            columns=output_data_columns
        )
    ], axis="columns")
)

loio_scores_df.to_csv(output_summary_file, index=False, compression="gzip", sep="\t")

print(loio_scores_df.shape)
loio_scores_df.head()

(26280, 13)


,Metadata_DNA,Model_type,Dataset_type,Mitocheck_Phenotypic_Class,Model_Feature_Type,Balance_type,Average_Rank,Average_P_Value,Min_IQR_Rank,Max_IQR_Rank,Min_IQR_P_Value,Max_IQR_P_Value,Count
0,LT0003_40/LT0003_40_149_83.tif,final,ic,Polylobed,CP,balanced,2.333333,0.651796,1.0,3.0,0.482906,0.977251,3
1,LT0003_40/LT0003_40_149_83.tif,final,ic,Polylobed,CP,unbalanced,2.333333,0.653764,1.0,3.0,0.485112,0.980265,3
2,LT0003_40/LT0003_40_149_83.tif,final,ic,Polylobed,CP_and_DP,balanced,2.000000,0.652280,1.0,2.5,0.484353,0.977610,3
3,LT0003_40/LT0003_40_149_83.tif,final,ic,Polylobed,CP_and_DP,unbalanced,2.000000,0.652280,1.0,2.5,0.484353,0.977610,3
4,LT0003_40/LT0003_40_149_83.tif,final,ic,Polylobed,CP_areashape_only,balanced,2.666667,0.242945,1.5,3.5,0.078920,0.364122,3


## Get average ranks and p value per phenotype

- Per model type (final vs. shuffled)
- Per illumination correction function (IC vs. No-IC)
- Per Phenotype
- Per Feature Space

(i.e., not on a per-image basis)

In [7]:
# Calculate average rank for each phenotype
rank_groups.remove("Metadata_DNA")  # Remove the per image to group on

avg_ranks = (
    loio_df.groupby(rank_groups)
    .apply(compute_avg_rank_and_pvalue)
    .reset_index()
)

avg_ranks.columns = rank_groups + ["Average_Scores"]

loio_scores_df = (
    pd.concat([
        avg_ranks.drop(columns="Average_Scores"),
        pd.DataFrame(
            avg_ranks.Average_Scores.tolist(),
            columns=output_data_columns
        )
    ], axis="columns")
)

loio_scores_df.to_csv(output_summary_phenotype_file, index=False, compression="gzip", sep="\t")

print(loio_scores_df.shape)
loio_scores_df.head()

(600, 12)


,Model_type,Dataset_type,Mitocheck_Phenotypic_Class,Model_Feature_Type,Balance_type,Average_Rank,Average_P_Value,Min_IQR_Rank,Max_IQR_Rank,Min_IQR_P_Value,Max_IQR_P_Value,Count
0,final,ic,ADCCM,CP,balanced,2.273684,0.423767,1.0,3.0,0.041222,0.717216,95
1,final,ic,ADCCM,CP,unbalanced,2.263158,0.434702,1.0,3.0,0.039917,0.740537,95
2,final,ic,ADCCM,CP_and_DP,balanced,2.252632,0.496073,1.0,3.0,0.090981,0.825156,95
3,final,ic,ADCCM,CP_and_DP,unbalanced,2.252632,0.496073,1.0,3.0,0.090981,0.825156,95
4,final,ic,ADCCM,CP_areashape_only,balanced,5.494737,0.101327,3.0,8.0,0.025864,0.142731,95
